**Importing modules**

In [16]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import string 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

# Tensorflow modules
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.text import Tokenizer


**Creating a function to clean all the data**

In [17]:
def clean_data(text):
    text=str(text)
    text=''.join([i for i in text if i not in string.punctuation]) 
    text=text.encode('utf-8').decode('ascii','ignore')
    return text

**Building the custom sequence generator class**

In [18]:
class Data_generator(Sequence):
    def __init__(self,texts,labels,tokenizer,max_sequence_len,batch_size=512):
        self.texts=texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_sequence_len = max_sequence_len
        self.batch_size = batch_size
        self.indices = np.arange(len(self.texts)) 

    def __len__(self):
        return int(np.ceil(len(self.texts)/self.batch_size)) 

    def __getitem__(self,idx):
        batch_indices=self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_texts=[self.texts[i] for i in batch_indices]
        batch_labels=[self.labels[i] for i in batch_indices] 

        sequences=self.tokenizer.texts_to_sequences(batch_texts)
        padded=pad_sequences(sequences,maxlen=self.max_sequence_len,padding='post',truncating='post')

        return np.array(padded),np.array(batch_labels) 

**Loading the dataset**

In [19]:
data=pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

# Cleaning the data
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0}) #encoding positive and negative to make it easier in training
data['review']=data['review'].apply(clean_data)


**Splitting into training and testing data**

In [20]:
train_val_data,test_data=train_test_split(data,test_size=5000,random_state=42,stratify=data['sentiment'])
train_data,val_data=train_test_split(train_val_data,test_size=5000,random_state=42,stratify=train_val_data['sentiment'])

**Fitting the tokenzier**

In [21]:
max_words=20000
max_len=200

tokenizer=Tokenizer(num_words=max_words,oov_token="<OOV>")
tokenizer.fit_on_texts(train_val_data['review'])

**Building the 3 Models**

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense,Dropout,LSTM,GRU,BatchNormalization

**SimpleRNN**

In [23]:
model_1=Sequential()
model_1.add(Embedding(max_words,128,input_length=max_len))
model_1.add(SimpleRNN(128,return_sequences=True))
model_1.add(Dropout(0.3))
model_1.add(SimpleRNN(64,return_sequences=False))
model_1.add(BatchNormalization())
model_1.add(Dense(64))
model_1.add(Dropout(0.4))
model_1.add(Dense(1,activation='sigmoid'))

model_1.build(input_shape=(None, max_len))
model_1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_1.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 200, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 200, 128)       │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 200, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,609,729 (9.96 MB)

 Trainable params: 2,609,601 (9.95 MB)

 Non-trainable params: 128 (512.00 B)

<hr>

**LSTM**

In [24]:
model_2=Sequential()
model_2.add(Embedding(max_words,128,input_length=max_len))
model_2.add(LSTM(128,return_sequences=True))
model_2.add(Dropout(0.3))
model_2.add(LSTM(64,return_sequences=False))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.4))

model_2.add(Dense(64))
model_2.add(Dense(1,activation='sigmoid'))

model_2.build(input_shape=(None, max_len))
model_2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_2.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 200, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 200, 128)       │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 200, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,745,473 (10.47 MB)

 Trainable params: 2,745,345 (10.47 MB)

 Non-trainable params: 128 (512.00 B)

<hr>

**GRU**

In [25]:
model_3=Sequential()
model_3.add(Embedding(max_words,128,input_length=max_len))
model_3.add(GRU(128,return_sequences=True))
model_3.add(Dropout(0.3))
model_3.add(GRU(64,return_sequences=False))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.3))
model_3.add(Dense(64))
model_3.add(Dense(1,activation='sigmoid'))

model_3.build(input_shape=(None, max_len))
model_3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_3.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 200, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 200, 128)       │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 200, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,700,801 (10.30 MB)

 Trainable params: 2,700,673 (10.30 MB)

 Non-trainable params: 128 (512.00 B)

**Using the data generators**

In [26]:
train_gen = Data_generator(
    train_data['review'].tolist(),
    train_data['sentiment'].tolist(),
    tokenizer,
    max_len,
    batch_size=512
)

test_gen = Data_generator(
    test_data['review'].tolist(),
    test_data['sentiment'].tolist(),
    tokenizer,
    max_len,
    batch_size=512
) 

val_gen= Data_generator(
    val_data['review'].tolist(),
    val_data['sentiment'].tolist(),
    tokenizer,
    max_len,
    batch_size=512
)


**Creating callbacks**

In [29]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

early_stopping=EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# checkpoint=ModelCheckpoint(
#     'best_model.keras',
#     monitor='val_loss',
#     save_best_only=True,
#     mode='min',
#     verbose=1
# )

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,      
    patience=3,      
    min_lr=1e-6,     
    verbose=1
)
callbacks=[early_stopping,reduce_lr]

**Training the model**

In [30]:
models = [
    (model_1, 'SimpleRNN'),
    (model_2, 'LSTM'),
    (model_3, 'GRU')
]

histories = {}
for m, name in models:
    print(f"\nTraining {name}...\n\n")
    histories[name] = m.fit(
        train_gen,
        validation_data=val_gen,
        epochs=50,
        callbacks=callbacks,
        verbose=1
    )
    m.save(f"{name}.keras")

Training SimpleRNN...
Epoch 1/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.4976 - loss: 0.6936 - val_accuracy: 0.5090 - val_loss: 0.6930 - learning_rate: 2.0000e-04
Epoch 2/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.5034 - loss: 0.6933 - val_accuracy: 0.5090 - val_loss: 0.6930 - learning_rate: 2.0000e-04
Epoch 3/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.4986 - loss: 0.6937 - val_accuracy: 0.5092 - val_loss: 0.6930 - learning_rate: 2.0000e-04
Epoch 4/50
78/79 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5067 - loss: 0.6932
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.5066 - loss: 0.6932 - val_accuracy: 0.4910 - val_loss: 0.6932 - learning_rate: 2.0000e-04
Epoch 5/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.4948 - loss: 0.6938 - val_accuracy: 0.5090 - val_loss: 0.6931 - learning_rate: 4.0000e-05
Epoch 6/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step -

In [ ]:
for m, name in models:
    loss, acc = m.evaluate(test_gen, verbose=1)
    print(f"{name} Test Accuracy: {acc:.4f} | Test Loss: {loss:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.4915 - loss: 0.6937
SimpleRNN Test Accuracy: 0.4976 | Test Loss: 0.6934
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.8666 - loss: 0.3321
LSTM Test Accuracy: 0.8654 | Test Loss: 0.3300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.8765 - loss: 0.3084
GRU Test Accuracy: 0.8716 | Test Loss: 0.3087
